In [1]:
%pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00


In [2]:
!pip install gTTS langdetect SpeechRecognition pygame pydub --quiet
!apt-get install ffmpeg libespeak1 -y > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.1 MB/s eta 0:00:00


In [3]:
import streamlit as st
from PIL import Image
import numpy as np
import tensorflow as tf
import json
import os
import tempfile
import speech_recognition as sr
from pydub import AudioSegment


In [4]:
treatment_guide = {
    "Apple_Scab": "Apply fungicides like captan or mancozeb. Remove infected leaves and ensure good air circulation.",
    "Black_Rot": "Remove infected fruit and apply fungicide sprays like thiophanate-methyl during bloom.",
    "Cedar_Apple_Rust": "Use fungicides like myclobutanil and remove nearby juniper trees if possible.",

    "Healthy": "No treatment needed. Maintain good hygiene and monitor regularly.",

    "Cercospora_Leaf_Spot": "Use resistant hybrids and apply fungicides like azoxystrobin.",
    "Common_Rust": "Apply fungicides like propiconazole. Plant resistant varieties.",
    "Northern_Leaf_Blight": "Use fungicides and rotate crops to reduce fungal spores.",

    "Esca_Black_Measles": "Remove infected vines and use fungicides. Avoid overwatering.",
    "Leaf_Blight": "Use fungicides like thiophanate-methyl. Prune infected leaves.",

    "Leaf_Scorch": "Use fungicides like captan. Avoid wet foliage and overcrowding.",

    "Powdery_Mildew": "Spray sulfur or neem oil. Remove infected leaves and improve airflow.",
    "Early_Blight": "Use fungicides like chlorothalonil or mancozeb. Rotate crops annually.",
    "Late_Blight": "Destroy infected plants. Apply protective fungicides like mancozeb or cymoxanil.",

    "Bacterial_Spot": "Apply copper-based sprays and avoid overhead irrigation.",

    "Septoria_Leaf_Spot": "Remove infected leaves and use fungicides like maneb.",
    "Yellow_Leaf_Curl_Virus": "Remove infected plants and control whiteflies with insecticides."
}
def get_treatment(disease_name):
    return treatment_guide.get(disease_name, "⚠️ No specific treatment found. Please consult an expert.")

In [5]:
MODEL_PATH = "/content/plant_disease_model (1).h5"
CLASS_JSON = "/content/class_names (1).json"

if os.path.exists(MODEL_PATH):
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(CLASS_JSON, "r") as f:
        class_names = json.load(f)
else:
    model = None
    class_names = []

In [6]:
st.set_page_config(page_title="RootIntel", layout="wide")

st.markdown("""
    <style>
    body {
        background: linear-gradient(to right, #e6f9f0, #e0f7fa);
    }
    .title-container {
        text-align: center;
        padding: 20px;
        background: linear-gradient(to right, #a5f3b9, #a0e8f0);
        border-radius: 12px;
        margin-bottom: 20px;
    }
    </style>
""", unsafe_allow_html=True)

2025-06-28 04:53:59.544 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:53:59.546 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:53:59.902 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-28 04:53:59.903 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:53:59.904 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [7]:
st.markdown("""
<div class="title-container">
    <h1>🌱 RootIntel</h1>
    <h4>AI-powered website for plant disease detection for gardeners</h4>
</div>
""", unsafe_allow_html=True)

col1, col2 = st.columns(2)

2025-06-28 04:54:09.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:09.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:09.481 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:09.482 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:09.483 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:09.483 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [8]:
with col1:
    st.subheader("📷 Upload Plant Photo for AI Detection")
    uploaded_file = st.file_uploader("Upload Leaf Image", type=["jpg", "jpeg", "png"])

    if uploaded_file and model:
        image = Image.open(uploaded_file).resize((224, 224))
        st.image(image, caption="Uploaded Leaf", use_column_width=True)

        img_array = tf.keras.utils.img_to_array(image) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model.predict(img_array)
        pred_index = np.argmax(pred)
        pred_class = class_names[pred_index]
        confidence = pred[0][pred_index]

        treatment = get_treatment(pred_class)

        st.success(f"🦠 Detected Disease: **{pred_class}**")
        st.markdown(f"""
        <div style='background-color: #e8f5e9; padding: 10px; border-radius: 10px;'>
        <strong>💊 Treatment:</strong> {treatment}
        </div>
        """, unsafe_allow_html=True)
        st.caption(f"Confidence: {confidence*100:.2f}%")

2025-06-28 04:54:15.302 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.303 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.305 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.306 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.307 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.309 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:15.310 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [9]:
with col2:
    st.subheader("🎙 Describe Symptoms (Voice Input)")
    audio_file = st.file_uploader("Upload a .wav file", type=["wav"])

    if audio_file:
        recognizer = sr.Recognizer()
        audio_path = tempfile.NamedTemporaryFile(delete=False).name
        with open(audio_path, "wb") as f:
            f.write(audio_file.read())
        with sr.AudioFile(audio_path) as source:
            audio = recognizer.record(source)
            try:
                result = recognizer.recognize_google(audio)
                st.success(f"You said: {result}")
            except:
                st.warning("Could not understand your voice.")

    text_input = st.text_area("Or type symptoms here:", placeholder="e.g., My tomato has brown spots on the leaves")

    if st.button("Get AI Plant Diagnosis"):
        st.info("🧠 Text/voice-based symptom analysis will be available in the next version.")

2025-06-28 04:54:24.243 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.246 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.248 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.249 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-28 04:54:24.250 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [10]:
!pip install streamlit pyngrok speechrecognition pydub


In [16]:
!ngrok config add-authtoken 2z60ksVgh9tOrkI6V0cgFvidIRa_2No9Vsd5uaaTfYyN9TFA7



Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [14]:
!tail -f /content/log.txt


tail: cannot open '/content/log.txt' for reading: No such file or directory
tail: no files remaining


In [18]:
from pyngrok import ngrok
ngrok.kill()

In [19]:
from pyngrok import ngrok

# Kill any previous tunnels
ngrok.kill()

# 🔁 Start Streamlit in background (first!)
!streamlit run rootintel_ui.py &>/content/log.txt &

# ✅ Then start ngrok to forward port 8501
public_url = ngrok.connect(8501)
print(f"🌐 RootIntel is live at: {public_url}")


🌐 RootIntel is live at: NgrokTunnel: "https://cf3f-34-75-165-233.ngrok-free.app" -> "http://localhost:8501"
